In [1]:
from sklearn.model_selection import train_test_split

In [2]:
# -*- coding: utf-8 -*-
"""Hackathon.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/19Y6nCWGiR6U0cKHUCMEo1XbPKbVr2xRx

# Importing Libraries
"""

import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

import nltk
from wordcloud import WordCloud, STOPWORDS

data = pd.read_csv("Tweets.csv")

data.shape

data.describe()

data = data.loc[:,["text","airline_sentiment"]]

for k,tx in enumerate(data["text"]):
    li = " ".join([word for word in tx.split() if 'http' not in word and not word.startswith('@')and word != 'RT'])
    data.loc[k,"text"] = li

Y = data["airline_sentiment"].apply(lambda x: 0 if x == "negative"  else (1 if (x == "nuetral") else 2))

data["airline_sentiment"].value_counts()

"""# Text mining
1. Removing Stopword, Stemming/lemmatizing, Removing all the symbols
"""

punc = '''!()-[]{};:'"\, <>./?@#$%^&*_~'''

for k,tx in enumerate(data["text"]):
    res = re.sub(r'[^\w\s]', '', tx)
    data.loc[k,"text"] = ''.join([i.lower() for i in res if not i.isdigit()])

nltk.download("stopwords")

from nltk.corpus import stopwords
stops = stopwords.words('english')

for k,tx in enumerate(data["text"]):
    li = " ".join([word for word in tx.split() if word not in stops])
    data.loc[k,"text"] = li

nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
ltz = WordNetLemmatizer()

for k,tx in enumerate(data["text"]):
    li = " ".join([ltz.lemmatize(word) for word in tx.split()])
    data.loc[k,"text"] = li

X = data["text"]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
from gensim.models import Word2Vec
from gensim.test.utils import get_tmpfile

In [4]:
sentences = [[] for k in range(len(data["text"]))]
for k,tx in enumerate(data["text"]):
    sentences[k]  = tx.split()

In [5]:
model = Word2Vec(sentences, min_count=1)
# summarize the loaded model
print(model)
# summarize vocabulary
words = list(model.wv.vocab)
print(words)
# access vector for one word
print(model['avoid'])
# save model
model.save('model.bin')
file_name = "word2vec_embeddings.txt"
model.wv.save_word2vec_format(file_name, binary = False)
# load model
new_model = Word2Vec.load('model.bin')

Word2Vec(vocab=11535, size=100, alpha=0.025)
['said', 'plus', 'youve', 'added', 'commercial', 'experience', 'tacky', 'didnt', 'today', 'must', 'mean', 'need', 'take', 'another', 'trip', 'really', 'aggressive', 'blast', 'obnoxious', 'entertainment', 'guest', 'face', 'amp', 'little', 'recourse', 'big', 'bad', 'thing', 'seriously', 'would', 'pay', 'flight', 'seat', 'playing', 'flying', 'va', 'yes', 'nearly', 'every', 'time', 'fly', 'vx', 'ear', 'worm', 'wont', 'go', 'away', 'missed', 'prime', 'opportunity', 'men', 'without', 'hat', 'parody', 'well', 'didntbut', 'amazing', 'arrived', 'hour', 'early', 'youre', 'good', 'know', 'suicide', 'second', 'leading', 'cause', 'death', 'among', 'teen', 'lt', 'pretty', 'graphic', 'much', 'better', 'minimal', 'iconography', 'great', 'deal', 'already', 'thinking', 'nd', 'havent', 'even', 'gone', 'st', 'yet', 'p', 'im', 'fabulous', 'seductive', 'sky', 'u', 'stress', 'travel', 'thanks', 'sfopdx', 'schedule', 'still', 'mia', 'excited', 'first', 'cross', 'co

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:254: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [6]:
def onehot(arr):
    li = np.zeros((3,len(arr)))
    for i,k in enumerate(arr):
        hot = [0,0,0]
        # print(k)
        hot[k] = 1
        li[:,i] = hot
    return li
# Y_train = onehot(Y_train)
# Y_test = onehot(Y_test)

# Y_test = Y_test.transpose()
# Y_train = Y_train.transpose()

In [7]:
dic = {}
for tx in data["text"]:
    for word in tx.split():
        if word not in dic:
            dic[word] = len(dic)

In [8]:
embedding_dict = {}
for k in dic.keys():
    embedding_dict[k] = model[k]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  This is separate from the ipykernel package so we can avoid doing imports until


In [9]:
Max_str_len = 64
Vec_x = np.zeros((len(data["text"]),100,64))
for k,tx in enumerate(data["text"]):
    for i,word in enumerate(tx.split()):
        Vec_x[k,:,i] = model[word]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """


In [10]:
X_train, X_test, Y_train, Y_test = train_test_split(Vec_x,Y,test_size = 0.1, random_state= 10)

In [11]:
from  tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import ModelCheckpoint

In [12]:
len(data["text"])

14640

In [40]:
token = Tokenizer()
token.fit_on_texts(data["text"])

In [41]:
max_len = max([len(s.split()) for s in data["text"]])

In [42]:
vocab_size = len(token.word_index) + 1

In [43]:
X = token.texts_to_sequences(data["text"])
X_pad = pad_sequences(X,maxlen=max_len, padding="post")

In [49]:
X_train,X_test,Y_train,Y_test = train_test_split(X_pad,Y, test_size=0.1, random_state = 10)

In [50]:
Y_train = onehot(Y_train)
Y_test = onehot(Y_test)

Y_test = Y_test.transpose()
Y_train = Y_train.transpose()


In [51]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Embedding,LSTM

In [52]:
model = Sequential()
model.add(Embedding(vocab_size,100 , input_length=max_len))
model.add(LSTM(10))
model.add(Dropout(0.7))
model.add(Dense(3,activation='sigmoid'))
checkpoint = ModelCheckpoint('model.h5', verbose=1, monitor='val_accuracy',save_best_only=True, mode='auto')
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 21, 100)           1153100   
_________________________________________________________________
lstm_4 (LSTM)                (None, 10)                4440      
_________________________________________________________________
dropout_4 (Dropout)          (None, 10)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 3)                 33        
Total params: 1,157,573
Trainable params: 1,157,573
Non-trainable params: 0
_________________________________________________________________
None


In [53]:
history = model.fit(X_train,Y_train, epochs=10, callbacks=[checkpoint],validation_data=(X_test,Y_test), batch_size = 64)

Epoch 1/10
204/206 [============================>.] - ETA: 0s - loss: 0.7917 - accuracy: 0.6029
Epoch 00001: val_accuracy improved from -inf to 0.61612, saving model to model.h5
206/206 [==============================] - 5s 25ms/step - loss: 0.7908 - accuracy: 0.6035 - val_loss: 0.5622 - val_accuracy: 0.6161
Epoch 2/10
205/206 [============================>.] - ETA: 0s - loss: 0.6111 - accuracy: 0.7113
Epoch 00002: val_accuracy improved from 0.61612 to 0.80943, saving model to model.h5
206/206 [==============================] - 5s 24ms/step - loss: 0.6108 - accuracy: 0.7114 - val_loss: 0.5149 - val_accuracy: 0.8094
Epoch 3/10
206/206 [==============================] - ETA: 0s - loss: 0.5413 - accuracy: 0.7684
Epoch 00003: val_accuracy did not improve from 0.80943
206/206 [==============================] - 5s 23ms/step - loss: 0.5413 - accuracy: 0.7684 - val_loss: 0.4776 - val_accuracy: 0.7883
Epoch 4/10
206/206 [==============================] - ETA: 0s - loss: 0.4806 - accuracy: 0.826

In [54]:
token = Tokenizer()
token.fit_on_texts(data["text"])
max_len = max([len(s.split()) for s in data["text"]])
vocab_size = len(token.word_index) + 1
X = token.texts_to_sequences(data["text"])
X_pad = pad_sequences(X,maxlen=max_len, padding="post")

In [55]:
word_index = len(token.word_index)
num_words = word_index + 1
embeding_mat = np.zeros((num_words,100))
for word,i in token.word_index.items():
    if(word in embedding_dict):
        embeding_mat[i] = embedding_dict[word]

In [56]:
from tensorflow.keras.initializers import Constant

In [57]:
model = Sequential()
embed = Embedding(num_words,100,embeddings_initializer=Constant(embeding_mat),input_length=max_len,trainable = True)
model.add(embed)
model.add(LSTM(50))
model.add(Dense(10))
model.add(Dense(3,activation='softmax'))
checkpoint = ModelCheckpoint('model_word2vec.h5', verbose=1, monitor='val_accuracy',save_best_only=True, mode='auto')
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 21, 100)           1153100   
_________________________________________________________________
lstm_5 (LSTM)                (None, 50)                30200     
_________________________________________________________________
dense_5 (Dense)              (None, 10)                510       
_________________________________________________________________
dense_6 (Dense)              (None, 3)                 33        
Total params: 1,183,843
Trainable params: 1,183,843
Non-trainable params: 0
_________________________________________________________________
None


In [58]:
history = model.fit(X_train,Y_train, epochs=10, callbacks=[checkpoint],validation_data=(X_test,Y_test), batch_size = 64)

Epoch 1/10
205/206 [============================>.] - ETA: 0s - loss: 0.5606 - accuracy: 0.7319
Epoch 00001: val_accuracy improved from -inf to 0.80533, saving model to model_word2vec.h5
206/206 [==============================] - 8s 39ms/step - loss: 0.5602 - accuracy: 0.7322 - val_loss: 0.4400 - val_accuracy: 0.8053
Epoch 2/10
205/206 [============================>.] - ETA: 0s - loss: 0.3566 - accuracy: 0.8482
Epoch 00002: val_accuracy improved from 0.80533 to 0.81967, saving model to model_word2vec.h5
206/206 [==============================] - 8s 36ms/step - loss: 0.3564 - accuracy: 0.8484 - val_loss: 0.3995 - val_accuracy: 0.8197
Epoch 3/10
205/206 [============================>.] - ETA: 0s - loss: 0.2639 - accuracy: 0.8936
Epoch 00003: val_accuracy improved from 0.81967 to 0.82582, saving model to model_word2vec.h5
206/206 [==============================] - 8s 37ms/step - loss: 0.2639 - accuracy: 0.8934 - val_loss: 0.4130 - val_accuracy: 0.8258
Epoch 4/10
205/206 [=================

# Unviersal Encoding Based Classification

In [59]:
from absl import logging

import tensorflow as tf

import tensorflow_hub as hub
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import seaborn as sns

module_url = "https://tfhub.dev/google/universal-sentence-encoder/4" 
model = hub.load(module_url)
print ("module %s loaded" % module_url)
def embed(input):
  return model(input)

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


module https://tfhub.dev/google/universal-sentence-encoder/4 loaded


In [60]:
arr_embed = embed(data["text"])

In [61]:
arr_embed.shape

TensorShape([14640, 512])

In [62]:
X_train, X_test, Y_train, Y_test = train_test_split(np.array(arr_embed),Y,test_size = 0.1, random_state= 10)

In [63]:
X_train.shape

(13176, 512)

In [64]:
Y_train = onehot(Y_train)
Y_test = onehot(Y_test)

Y_test = Y_test.transpose()
Y_train = Y_train.transpose()

In [65]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout
model = Sequential()
model.add(Dense(300, input_dim=X_train.shape[1], activation='relu'))
model.add(Dropout(0.9))
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.7))
model.add(Dense(10,activation="relu"))
model.add(Dense(3,activation= "sigmoid"))
checkpoint = ModelCheckpoint('use_model.h5', verbose=1, monitor='val_accuracy',save_best_only=True, mode='auto')
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

history = model.fit(X_train, Y_train,validation_data = (X_test,Y_test), epochs=20,callbacks=[checkpoint], batch_size=32)


Epoch 1/20
409/412 [============================>.] - ETA: 0s - loss: 0.6404 - accuracy: 0.6783
Epoch 00001: val_accuracy improved from -inf to 0.81284, saving model to use_model.h5
412/412 [==============================] - 2s 4ms/step - loss: 0.6393 - accuracy: 0.6791 - val_loss: 0.4126 - val_accuracy: 0.8128
Epoch 2/20
404/412 [============================>.] - ETA: 0s - loss: 0.4458 - accuracy: 0.8038
Epoch 00002: val_accuracy improved from 0.81284 to 0.82923, saving model to use_model.h5
412/412 [==============================] - 2s 4ms/step - loss: 0.4460 - accuracy: 0.8035 - val_loss: 0.3976 - val_accuracy: 0.8292
Epoch 3/20
407/412 [============================>.] - ETA: 0s - loss: 0.4222 - accuracy: 0.8164
Epoch 00003: val_accuracy did not improve from 0.82923
412/412 [==============================] - 2s 4ms/step - loss: 0.4221 - accuracy: 0.8165 - val_loss: 0.3972 - val_accuracy: 0.8238
Epoch 4/20
405/412 [============================>.] - ETA: 0s - loss: 0.4091 - accuracy: 

In [66]:
model.load_weights("use_model.h5")
model.evaluate(X_test,Y_test, verbose=True)

46/46 [==============================] - 0s 2ms/step - loss: 0.3754 - accuracy: 0.8388


[0.375353068113327, 0.8387978076934814]